## Eulerian Fluid Simulation
*still sensors that never moves*
- Eulerian representation uses still sensors in space, usually arranged in a
 regular grid/triangular mesh.
- A bit of math - but not too much.

### Overview
#### Material Derivatives: Lagrangian v.s. Eulerian (材料导数，物质导数)

\begin{equation*}
\frac{D}{Dt} := \frac{\partial}{\partial t}+{\rm u}\cdot\nabla
\end{equation*}

E.g.,<br>
\begin{equation*}\begin{array}{rcl}
\displaystyle\frac{DT}{Dt} & = & \displaystyle\frac{\partial T}{\partial t}+{\rm u}\cdot\nabla T \\
\displaystyle\frac{D{\rm u}_{x}}{Dt} & = & \displaystyle\frac{\partial {\rm u}_{x}}{\partial t}+{\rm u}\cdot\nabla {\rm u}_{x}
\end{array}\end{equation*}<br>
${\rm u}$: material (fluid) velocity. Many other names: Advective/Lagrangian/particle derivative.
Intuitively, change of physical quantity on a piece of material =
1. change due to time $\frac{\partial}{\partial t}$ (Eulerian).
2. change due to material movement ${\rm u}\cdot\nabla$.

#### (Incompressible) Navier-Stokes equations
\begin{array}{rcl}
\rho\displaystyle\frac{D{\rm u}}{Dt}&=&-\nabla p+\mu\nabla^{2}{\rm u}+\rho g \\
\displaystyle\frac{D{\rm u}}{Dt}&=&-\displaystyle\frac{1}{\rho}\nabla p+\nu\nabla^{2}{\rm u}+g \\
\nabla \cdot {\rm u}&=&0
\end{array}<br>
$\mu$: dynamic viscosity; $\nu=\frac{\mu}{\rho}$: kinematic viscosity.<br>
In graphics we usually drop **viscosity** except for highly viscous materials (e.g., honey)

#### Operator splitting
\begin{array}{rclc}
\displaystyle\frac{D{\rm u}}{Dt}&=&-\displaystyle\frac{1}{\rho}\nabla p+g,&\nabla \cdot {\rm u}=0
\end{array}<br>
Split the equations above into three parts:<br>
\begin{array}{rcllr}
\displaystyle\frac{D{\rm u}}{Dt}&=&0,\displaystyle\frac{D\alpha}{Dt}=0&(\mathbf{advection})&(1) \\
\displaystyle\frac{\partial{\rm u}}{\partial t}&=&g&({\rm external forces, optional})&(2) \\
\displaystyle\frac{\partial{\rm u}}{\partial t}&=&-\displaystyle\frac{1}{\rho}\nabla p\qquad s.t.\quad\nabla \cdot {\rm u}=0&(\mathbf{projection})&(3)
\end{array}<br>
$\alpha$: any physical property (temperature, color, smoke density etc.)

#### Eulerian ﬂuid simulation cycle
Time discretization with splitting: for each time step,<br>
1. Advection: “move” the ﬂuid feld. Solve ${\rm u}^{*}$ using  ${\rm u}^{t}$<br>
\begin{equation}
\displaystyle\frac{D{\rm u}}{Dt}=0,\displaystyle\frac{D\alpha}{Dt}=0
\end{equation}
2. External forces (optional): evaluate ${\rm u}^{**}$ using ${\rm u}^{*}$<br>
\begin{equation}
\displaystyle\frac{\partial{\rm u}}{\partial t}=g
\end{equation}
3. Projection: make velocity field ${\rm u}^{t+1}$ dicergence-free based on ${\rm u}^{**}$<br>
\begin{equation}
\displaystyle\frac{\partial{\rm u}}{\partial t}=-\displaystyle\frac{1}{\rho}\nabla p\qquad s.t.\quad\nabla \cdot{\rm u}^{t+1}=0
\end{equation}

### Grid
#### Spatial discretization using cell-centered grids
![fig](./cell-centered-grigs.png)<br>
Figure: ${\rm u}_{x},{\rm u}_{y},p$ are all stored at the center (orange) of cells

```python
import taichi as ti
n, m = 3, 3
u = ti.var(ti.f32, shape=(n, m)) # x-component of velocity
v = ti.var(ti.f32, shape=(n, m)) # y-component of velocity
p = ti.var(ti.f32, shape=(n, m)) # pressure
```

#### Spatial discretization using staggered grids
![fig](./staggered-grids.png)<br>
Figure: Red: ${\rm u}_{x}$; Green: ${\rm u}_{y}$; Orange: $p$.
```python
import taichi as ti
n, m = 3, 3
u = ti.var(ti.f32, shape=(n+1, m)) # x-component of velocity
v = ti.var(ti.f32, shape=(n, m+1)) # y-component of velocity
p = ti.var(ti.f32, shape=(n, m)) # pressure
```

#### Bilinear interpolation (双线性插值)
See PDF.

### Advection (对流)
#### Advection schemes
A trade-oﬀ between numerical viscosity, stability, performance and complexity:
- Semi-Lagrangian advection
- MacCormack/BFECC
- BiMocq
- Particle advection (PIC/FLIP/APIC/PolyPIC, later in this course)
- ...

#### Semi-Lagrangian advection